In [ ]:
import yaml
from sklearn.metrics import classification_report

from app.pool_classifiers import get_all_classifiers
from src.cca import learning_algorithm
from src.dataset import Dataset
from src.elements import Matrix, Pool
from src.helpers import dict_results_to_dataframe

In [ ]:
with open("src/cca_config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
all_classifiers = get_all_classifiers()
dataset = Dataset(
    name=config["dataset"]["name"],
    train_clf_amount=config["dataset"]["train_amount"],
    train_cca_amount=config["dataset"]["train_amount"],
    test_amount=config["dataset"]["test_amount"],
)
x_train_clf, y_train_clf = dataset.get_train_clf_samples()
x_train_cca, y_train_cca = dataset.get_train_cca_samples()
x_test, y_test = dataset.get_test_samples()

#BALANCE OF SAMPLES
from src.helpers import count_items_from_predict

#Classes data train classifiers
y_train_clf_0 = count_items_from_predict(y_train_clf, 0)/len(y_train_clf)
y_train_clf_1 = count_items_from_predict(y_train_clf, 1)/len(y_train_clf)
print((y_train_clf_0, y_train_clf_1))

#Classes data train classifiers
y_train_cca_0 = count_items_from_predict(y_train_cca, 0)/len(y_train_cca)
y_train_cca_1 = count_items_from_predict(y_train_cca, 1)/len(y_train_cca)
print((y_train_cca_0, y_train_cca_1))

#Classes data train classifiers
y_test_0 = count_items_from_predict(y_test, 0)/len(y_test)
y_test_1 = count_items_from_predict(y_test, 1)/len(y_test)
print((y_test_0, y_test_1))

In [ ]:
pool = Pool(all_classifiers)
pool.fit_all(x_train_clf, y_train_clf)
pool.test_all(x_test, y_test)
pool.remove_classifier_one_class()
results = pool.get_results()

results_df = dict_results_to_dataframe(results)

In [ ]:
# pool.shuffle_classifiers()
pool.filter_by_f1(0.3)
print("Quantidade de classificadores: ", len(pool.classifiers))
print("Maior f1 score no pool: ", max([f.f1 for f in pool.classifiers]))
print("Maior acurácia score no pool: ", max([f.score for f in pool.classifiers]))
lista_classificadores_filtrados = [c.name for c in pool.classifiers]
pool.shuffle_classifiers()
matrix = Matrix(
    size=config["matrix"]["size"],
    pool=pool,
    init_enery=config["init_energy"],
    distance_neighborhood=config["matrix"]["distance"],
)

In [ ]:
learning_algorithm(
    matrix=matrix,
    pool=pool,
    sample_features=x_train_cca,
    sample_class=y_train_cca,
    distance=config["matrix"]["distance"],
    interactions=config["interactions"],
    init_energy=config["init_energy"],
)

In [ ]:
# [[cell.get_classifier().name for cell in matrix.get()[i]] for i in range(8)]
[[cell.get_energy() for cell in matrix.get()[i]] for i in range(4)]

In [ ]:
matrix_class = matrix.predict_matrix(x_test)


In [ ]:
print(classification_report(y_test, matrix_class, digits=3))
# results_df_matrix
# matrix_class_0 = count_items_from_predict(matrix_class, 0)/len(matrix_class)
# matrix_class_1 = count_items_from_predict(matrix_class, 1)/len(matrix_class)
print("Qtd módulos com defeito na base: ", count_items_from_predict(y_test, 1))
print("Qtd módulos preditos: ", count_items_from_predict(matrix_class, 1))

In [ ]:
matrix.get_results(["recall"])
# [count_items_from_predict(p["prediction"],1) for p in matrix.get_results(["prediction"])]

In [ ]:
[c.get_results(["recall"]) for c in pool.get_classifiers()]